# VRP Demo with SCIP
This file is intended to build the model for the CVRP using SCIP

In [1]:
from pyscipopt import Model, Pricer, SCIP_RESULT, SCIP_STAGE
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import networkx as nx
import random
import matplotlib.pyplot as plt
import numpy as np
import sys

from parse import parse

from cffi import FFI
ffi = FFI()
labelling_lib = ffi.dlopen("Labelling/labelling_lib.so")

funDefs = "void initGraph(unsigned num_nodes, unsigned* node_data, double* edge_data, const double capacity); void labelling(double const * dual, const bool farkas, const bool elementary, unsigned* result);"
ffi.cdef(funDefs, override=True)

In [8]:
# Create Graph
G = nx.complete_graph(10)
for (u, v) in G.edges():
    G.edges[u,v]['weight'] = random.randint(1,10)
    
for node in G.nodes():
    G.nodes()[node]['demand'] = random.randint(1,10)

G.graph['capacity'] = 20
# nx.draw(G)

In [6]:
# Create Simple Graph for correctnes
G = nx.complete_graph(4)
for (u, v) in G.edges():
    G.edges[u,v]['weight'] = 1
G.edges[1,2]['weight'] = 1

for node in G.nodes():
    G.nodes()[node]['demand'] = 2

G.graph['capacity'] = 4
# nx.draw(G,with_labels=True)

In [2]:
# Test instance E-n22-k4 provided by parser
G = parse("Instances/E/E-n22-k4.vrp")

PARSE: Minimum number of trucks is 4


In [3]:
class VRP(Model):
    def __init__(self,graph):
        super().__init__()
        
        self.original_graph = graph
        self.graph = graph.copy()
        self.vars = {}
        self.cons = []

In [4]:
class VRPPricer(Pricer): 
    def pricerinit(self):
        self.data['cons'] = [self.model.getTransformedCons(con) for con in self.model.cons]
        self.data['vars'] = {path:self.model.getTransformedVar(var) for (path,var) in self.model.vars.items()}
        
#         print(f" There are {len(self.model.getConss())} constraints in the model and {len(self.data['cons'])} of them are known to the pricer.")
        
        node_data = list(nx.get_node_attributes(self.model.graph,"demand").values())
        if not np.all(np.array(node_data[1:])):
           print("PRICER_PY: The demands of all nodes must be > 0.")
#         print(f"PRICER_PY: The demands are {node_data}")
        nodes_arr = ffi.cast("unsigned*", np.array(node_data).astype(np.uintc).ctypes.data)

        edges = nx.adjacency_matrix(self.model.graph,dtype=np.double).toarray()
        edges_arr = ffi.cast("double*", edges.ctypes.data)
        
        num_nodes = ffi.cast("unsigned",self.model.graph.number_of_nodes())
        
        capacity_ptr = ffi.cast("double",self.data['capacity'])
        labelling_lib.initGraph(num_nodes,nodes_arr,edges_arr, capacity_ptr)
    
    def pricerfarkas(self):
        dual = [self.model.getDualfarkasLinear(con) for con in self.data['cons']]
#         print(f"PRICER_PY: Farkas Values are {dual}")
        return self.labelling(dual, farkas=True)

    def pricerredcost(self):
        dual = [self.model.getDualsolLinear(con) for con in self.data['cons']]
#         print(f"PRICER_PY: Dual variables are {dual}")
        return self.labelling(dual)
    
    def labelling(self, dual,farkas=False, elementary=True):
        pointer_dual = ffi.cast("double*", np.array(dual,dtype=np.double).ctypes.data)
        
        # Should be capacity + 2, but I gave it a little more room
        # TODO: Possible improvement: result can be reused every time
        result = np.zeros(self.data['capacity'] + 10,dtype=np.uintc)
        result_arr = ffi.cast("unsigned*",result.ctypes.data)
        
        labelling_lib.labelling(pointer_dual, farkas, elementary, result_arr)
        
        if(result[0] == 1):
#             print("There are no paths with negative reduced costs")
            return {'result':SCIP_RESULT.SUCCESS}
        
        result_indices = np.insert(np.nonzero(result),0,0)
        result_indices = np.append(result_indices,0)
        path = tuple(result[result_indices])
        print(f"PY PRICER: Found path {path}")
        # Kann entfernt werden, Frage hat sich mit Lukas geklärt
        if path in self.data['vars'].keys():
            cost = self.model.getVarRedcost(self.data['vars'][path])
            if farkas:
                print(f"PY Farkaspricing: Path already exists. | {path}")
            else:
                print(f"PY Pricing: Path already exists. Reduced Cost {cost:.2f} | {path}")
            return {'result':SCIP_RESULT.SUCCESS}
        
        var = self.model.addVar(vtype="C",obj=nx.path_weight(self.model.graph,path,"weight"),pricedVar=True)
        weight = nx.path_weight(self.model.graph,path,"weight")
        
        counts = np.unique(path[1:-1], return_counts=True)
        for i, node in enumerate(counts[0]):
            self.model.addConsCoeff(self.data['cons'][node-1], var ,counts[1][i])

        self.model.addConsCoeff(self.data['cons'][-1], var, 1)
        self.data['vars'][tuple(path)] = var
        
        return {'result':SCIP_RESULT.SUCCESS}

In [ ]:
model = VRP(G)
num_vehicles = 4

# Create pricer
pricer = VRPPricer()
pricer.data = {}
pricer.data["capacity"] = G.graph['capacity']
pricer.data["num_vehicles"] = num_vehicles
model.includePricer(pricer, "pricer","does pricing")

# Create a valid set of variables and the constraints to it
for i in range(1,G.number_of_nodes()):
    #TODO: I should check, whether these paths are indeed feasible.
    path = (0,i,0)
    cost = nx.path_weight(G,path,"weight")
#     print(f"Do these costs make sense? {cost}")
    var = model.addVar(vtype="C",obj=cost)
    model.vars[path] = var
    cons = model.addCons(var == 1, name=f"node_{i}",modifiable=True)
    model.cons.append(cons)
    
# Add the convexity constraint, which limits the number of available vehicles
convexity_constraint = model.addCons(sum(model.vars.values()) <= num_vehicles, modifiable=True)
model.cons.append(convexity_constraint)

# model.hideOutput()
model.optimize()
model.hideOutput(quiet=False)
# model.printBestSol()
sol = model.getBestSol()

# Flushing should probably prevent the console output from SCIP mix up with the following print
sys.stdout.flush()
print("\n\nThe solution contains the following paths: ")
for path, var in pricer.data['vars'].items():
    if sol[var] > 0.1:
        print(f"{sol[var]} * {var}: {path}")

PRICER_C: Graph data successfully copied to C.
PY PRICER: Found path (0, 1, 2, 0)presolving:
presolving (1 rounds: 1 fast, 1 medium, 1 exhaustive):
 0 deleted vars, 0 deleted constraints, 0 added constraints, 0 tightened bounds, 0 added holes, 0 changed sides, 0 changed coefficients
 0 implications, 0 cliques
presolved problem has 21 variables (0 bin, 0 int, 0 impl, 21 cont) and 22 constraints
     22 constraints of type <linear>
Presolving Time: 0.00


 time | node  | left  |LP iter|LP it/n|mem/heur|mdpt |vars |cons |rows |cuts |sepa|confs|strbr|  dualbound   | primalbound  |  gap   | compl. 
  0.1s|     1 |     0 |    10 |     - |   689k |   0 |  22 |  22 |  22 |   0 |  0 |   0 |   0 |      --      |      --      |    Inf | unknown
PY PRICER: Found path (0, 2, 3, 0)
PY PRICER: Found path (0, 1, 3, 0)
PY PRICER: Found path (0, 1, 4, 0)
PY PRICER: Found path (0, 3, 4, 0)
PY PRICER: Found path (0, 1, 5, 0)
PY PRICER: Found path (0, 2, 4, 0)
PY PRICER: Found path (0, 1, 6, 0)
PY PRICER: 

PY PRICER: Found path (0, 3, 4, 9, 0)
PY PRICER: Found path (0, 15, 19, 0)
PY PRICER: Found path (0, 1, 10, 13, 0)
PY PRICER: Found path (0, 1, 11, 13, 0)
PY PRICER: Found path (0, 1, 5, 14, 0)
PY PRICER: Found path (0, 16, 19, 0)
PY PRICER: Found path (0, 1, 8, 14, 0)
PY PRICER: Found path (0, 1, 10, 14, 0)
PY PRICER: Found path (0, 3, 5, 8, 0)
PY PRICER: Found path (0, 1, 2, 15, 0)
PY PRICER: Found path (0, 1, 9, 14, 0)
PY PRICER: Found path (0, 3, 5, 9, 0)
PY PRICER: Found path (0, 1, 13, 14, 0)
PY PRICER: Found path (0, 3, 6, 8, 0)
PY PRICER: Found path (0, 1, 3, 15, 0)
PY PRICER: Found path (0, 1, 5, 15, 0)
PY PRICER: Found path (0, 1, 4, 15, 0)
PY PRICER: Found path (0, 1, 6, 15, 0)
PY PRICER: Found path (0, 1, 10, 15, 0)
PY PRICER: Found path (0, 1, 2, 16, 0)
PY PRICER: Found path (0, 1, 9, 15, 0)
PY PRICER: Found path (0, 1, 11, 14, 0)
PY PRICER: Found path (0, 1, 11, 15, 0)
PY PRICER: Found path (0, 3, 6, 9, 0)
PY PRICER: Found path (0, 1, 14, 15, 0)
PY PRICER: Found path (0, 

PY PRICER: Found path (0, 4, 6, 15, 0)
PY PRICER: Found path (0, 4, 9, 12, 0)
PY PRICER: Found path (0, 2, 18, 20, 0)
PY PRICER: Found path (0, 4, 8, 13, 0)
PY PRICER: Found path (0, 4, 7, 15, 0)
PY PRICER: Found path (0, 4, 8, 14, 0)
PY PRICER: Found path (0, 4, 10, 11, 0)
PY PRICER: Found path (0, 4, 8, 15, 0)
PY PRICER: Found path (0, 4, 6, 16, 0)
PY PRICER: Found path (0, 4, 9, 13, 0)
PY PRICER: Found path (0, 4, 10, 12, 0)
PY PRICER: Found path (0, 4, 9, 14, 0)
PY PRICER: Found path (0, 4, 11, 12, 0)
PY PRICER: Found path (0, 4, 9, 15, 0)
PY PRICER: Found path (0, 6, 8, 9, 0)
PY PRICER: Found path (0, 4, 10, 13, 0)
PY PRICER: Found path (0, 4, 7, 16, 0)
PY PRICER: Found path (0, 4, 10, 14, 0)
PY PRICER: Found path (0, 4, 11, 13, 0)
PY PRICER: Found path (0, 4, 10, 15, 0)
PY PRICER: Found path (0, 4, 12, 13, 0)
PY PRICER: Found path (0, 4, 8, 16, 0)
PY PRICER: Found path (0, 2, 17, 21, 0)
PY PRICER: Found path (0, 4, 11, 14, 0)
PY PRICER: Found path (0, 4, 10, 16, 0)
PY PRICER: Fou

PY PRICER: Found path (0, 1, 2, 4, 7, 0)
PY PRICER: Found path (0, 4, 11, 20, 0)
PY PRICER: Found path (0, 8, 9, 20, 0)
PY PRICER: Found path (0, 4, 14, 20, 0)
PY PRICER: Found path (0, 4, 16, 20, 0)
PY PRICER: Found path (0, 4, 14, 21, 0)
PY PRICER: Found path (0, 4, 15, 20, 0)
PY PRICER: Found path (0, 4, 20, 21, 0)
PY PRICER: Found path (0, 1, 2, 4, 8, 0)
PY PRICER: Found path (0, 1, 4, 5, 6, 0)
PY PRICER: Found path (0, 1, 3, 4, 7, 0)
PY PRICER: Found path (0, 1, 4, 6, 7, 0)
PY PRICER: Found path (0, 9, 11, 14, 0)
PY PRICER: Found path (0, 9, 10, 16, 0)
PY PRICER: Found path (0, 5, 12, 21, 0)
PY PRICER: Found path (0, 1, 2, 5, 7, 0)
PY PRICER: Found path (0, 1, 4, 5, 7, 0)
PY PRICER: Found path (0, 3, 15, 21, 0)
PY PRICER: Found path (0, 3, 18, 21, 0)
PY PRICER: Found path (0, 3, 16, 21, 0)
PY PRICER: Found path (0, 3, 19, 21, 0)
PY PRICER: Found path (0, 9, 11, 15, 0)
PY PRICER: Found path (0, 5, 18, 21, 0)
PY PRICER: Found path (0, 3, 20, 21, 0)
PY PRICER: Found path (0, 7, 12, 2

PY PRICER: Found path (0, 1, 4, 6, 10, 0)
PY PRICER: Found path (0, 1, 4, 9, 10, 0)
PY PRICER: Found path (0, 1, 4, 10, 11, 0)
PY PRICER: Found path (0, 1, 2, 12, 13, 0)
PY PRICER: Found path (0, 14, 15, 16, 0)
PY PRICER: Found path (0, 1, 5, 7, 10, 0)
PY PRICER: Found path (0, 1, 5, 6, 8, 0)
PY PRICER: Found path (0, 1, 7, 8, 10, 0)
PY PRICER: Found path (0, 1, 3, 7, 11, 0)
PY PRICER: Found path (0, 1, 6, 7, 9, 0)
PY PRICER: Found path (0, 1, 3, 11, 12, 0)
PY PRICER: Found path (0, 1, 4, 7, 11, 0)
PY PRICER: Found path (0, 1, 4, 11, 12, 0)
PY PRICER: Found path (0, 1, 5, 7, 11, 0)
PY PRICER: Found path (0, 1, 7, 9, 10, 0)
PY PRICER: Found path (0, 1, 6, 7, 11, 0)
PY PRICER: Found path (0, 13, 16, 17, 0)
PY PRICER: Found path (0, 15, 16, 17, 0)
PY PRICER: Found path (0, 1, 7, 8, 11, 0)
PY PRICER: Found path (0, 13, 16, 19, 0)
PY PRICER: Found path (0, 1, 7, 9, 11, 0)
PY PRICER: Found path (0, 13, 17, 18, 0)
PY PRICER: Found path (0, 1, 7, 10, 11, 0)
PY PRICER: Found path (0, 13, 17, 19

PY PRICER: Found path (0, 2, 4, 7, 8, 0)
PY PRICER: Found path (0, 1, 2, 18, 19, 0)
PY PRICER: Found path (0, 1, 9, 15, 16, 0)
PY PRICER: Found path (0, 1, 2, 3, 21, 0)
PY PRICER: Found path (0, 1, 5, 18, 20, 0)
PY PRICER: Found path (0, 1, 6, 18, 19, 0)
PY PRICER: Found path (0, 1, 8, 11, 18, 0)
PY PRICER: Found path (0, 1, 2, 6, 20, 0)
  257s|     1 |     0 |  1282 |     - |  3875k |   0 |1022 |  22 |  22 |   0 |  0 |   0 |   0 |      --      |      --      |    Inf | unknown
PY PRICER: Found path (0, 1, 7, 18, 19, 0)
PY PRICER: Found path (0, 1, 8, 15, 17, 0)
PY PRICER: Found path (0, 1, 8, 12, 18, 0)
PY PRICER: Found path (0, 1, 2, 4, 21, 0)
PY PRICER: Found path (0, 1, 2, 5, 21, 0)
PY PRICER: Found path (0, 1, 2, 6, 21, 0)
PY PRICER: Found path (0, 1, 2, 8, 21, 0)
PY PRICER: Found path (0, 1, 2, 11, 21, 0)
PY PRICER: Found path (0, 1, 2, 12, 21, 0)
PY PRICER: Found path (0, 1, 2, 16, 21, 0)
PY PRICER: Found path (0, 1, 2, 20, 21, 0)
PY PRICER: Found path (0, 1, 5, 7, 21, 0)
PY PRI

PY PRICER: Found path (0, 1, 14, 19, 20, 0)
PY PRICER: Found path (0, 1, 15, 17, 18, 0)
PY PRICER: Found path (0, 1, 15, 16, 20, 0)
PY PRICER: Found path (0, 2, 3, 6, 13, 0)
PY PRICER: Found path (0, 2, 3, 4, 14, 0)
PY PRICER: Found path (0, 2, 3, 10, 13, 0)
PY PRICER: Found path (0, 1, 15, 17, 20, 0)
PY PRICER: Found path (0, 2, 3, 4, 15, 0)
PY PRICER: Found path (0, 1, 16, 17, 20, 0)
PY PRICER: Found path (0, 1, 17, 18, 19, 0)
PY PRICER: Found path (0, 2, 3, 6, 14, 0)
PY PRICER: Found path (0, 2, 3, 4, 16, 0)
PY PRICER: Found path (0, 1, 17, 18, 20, 0)
PY PRICER: Found path (0, 2, 3, 4, 20, 0)
PY PRICER: Found path (0, 1, 15, 18, 19, 0)
PY PRICER: Found path (0, 2, 3, 6, 15, 0)
PY PRICER: Found path (0, 2, 3, 10, 14, 0)
PY PRICER: Found path (0, 2, 3, 6, 19, 0)
PY PRICER: Found path (0, 1, 16, 18, 20, 0)
PY PRICER: Found path (0, 2, 3, 6, 20, 0)
PY PRICER: Found path (0, 2, 3, 10, 15, 0)
PY PRICER: Found path (0, 1, 16, 17, 21, 0)
PY PRICER: Found path (0, 1, 16, 18, 21, 0)
PY PRICER

PY PRICER: Found path (0, 3, 10, 12, 14, 0)
PY PRICER: Found path (0, 4, 5, 7, 10, 0)
PY PRICER: Found path (0, 3, 12, 13, 14, 0)
PY PRICER: Found path (0, 3, 11, 12, 14, 0)
PY PRICER: Found path (0, 3, 11, 13, 14, 0)
PY PRICER: Found path (0, 3, 8, 14, 15, 0)
PY PRICER: Found path (0, 3, 9, 14, 15, 0)
PY PRICER: Found path (0, 3, 6, 15, 16, 0)
PY PRICER: Found path (0, 3, 6, 16, 19, 0)
PY PRICER: Found path (0, 3, 8, 15, 16, 0)
PY PRICER: Found path (0, 4, 6, 8, 9, 0)
PY PRICER: Found path (0, 3, 10, 14, 15, 0)
PY PRICER: Found path (0, 3, 10, 11, 16, 0)
PY PRICER: Found path (0, 4, 5, 10, 11, 0)
PY PRICER: Found path (0, 4, 6, 7, 10, 0)
PY PRICER: Found path (0, 3, 11, 14, 15, 0)
PY PRICER: Found path (0, 4, 7, 8, 9, 0)
PY PRICER: Found path (0, 3, 8, 16, 19, 0)
PY PRICER: Found path (0, 6, 7, 8, 9, 0)
PY PRICER: Found path (0, 3, 7, 11, 16, 0)
PY PRICER: Found path (0, 4, 7, 8, 10, 0)
PY PRICER: Found path (0, 3, 7, 13, 15, 0)
PY PRICER: Found path (0, 3, 7, 13, 16, 0)
PY PRICER: Fo